In [1]:
"""
Test Script

"""
import random
import copy
import numpy as np
from vwgym import VacuumWorld, Vectorise, StepWrapper
from vwgym.dqn import *
from torch.distributions import Categorical
from vwgym.init_utils import *

if torch.cuda.is_available():
    print('GPU Available:\t', True)
    device = 'cuda'
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

else:
    device = 'cpu'

GPU Available:	 True
GPU Available:	 True


In [2]:
model_path = 'vwgym/saved_model/vwgym_dqn_450_ckpt.pt'
checkpoint = torch.load(model_path)

env, input_shape = make_env(grid_size=3, num_env=1, vectorize=True)
env = env[0]
env.state()

<class 'vacuumworld.vw.Grid'>: size: 3, agents: 1, dirts: 6, 
location(coordinate=coord(x=1, y=0), agent=agent(name='A-1', colour='white', orientation='east'), dirt=dirt(name='D-1', colour='orange'))
location(coordinate=coord(x=2, y=0), agent=None, dirt=dirt(name='D-3', colour='orange'))
location(coordinate=coord(x=0, y=1), agent=None, dirt=dirt(name='D-4', colour='green'))
location(coordinate=coord(x=1, y=1), agent=None, dirt=dirt(name='D-5', colour='green'))
location(coordinate=coord(x=0, y=2), agent=None, dirt=dirt(name='D-2', colour='orange'))
location(coordinate=coord(x=2, y=2), agent=None, dirt=dirt(name='D-6', colour='green'))


<class 'vacuumworld.vw.Grid'>: size: 3, agents: 1, dirts: 6, 
location(coordinate=coord(x=1, y=0), agent=agent(name='A-1', colour='white', orientation='east'), dirt=dirt(name='D-1', colour='orange'))
location(coordinate=coord(x=2, y=0), agent=None, dirt=dirt(name='D-3', colour='orange'))
location(coordinate=coord(x=0, y=1), agent=None, dirt=dirt(name='D-4', colour='green'))
location(coordinate=coord(x=1, y=1), agent=None, dirt=dirt(name='D-5', colour='green'))
location(coordinate=coord(x=0, y=2), agent=None, dirt=dirt(name='D-2', colour='orange'))
location(coordinate=coord(x=2, y=2), agent=None, dirt=dirt(name='D-6', colour='green'))

In [3]:
n_actions = env.action_space.n
dq_net = dqn(input_shape, n_actions, device)
dq_net.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [4]:
dq_net

dqn(
  (fc1): Linear(in_features=22, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=4, bias=True)
)

In [5]:
with torch.no_grad():
    dq_net.eval()
    predictions = []

    x = env.reset()
    env.rw_dirts = env.dirts
    print(f'Dirts Present..:{env.rw_dirts}')
        
    x = torch.from_numpy(x.reshape(1, -1)).float().to(device)
    step = 0
    prev_action = []
    
    for __ in range(5000):
        action = dq_net(x)
        a_t = action.max(1)[1].view(1, 1) #take_action(action_probs, 10000, greedy=True)
        x, reward, done, ep_info = env.step(a_t.item())#take_step(a_t, env, device)
        print(x, reward, done)
        x = torch.from_numpy(x.reshape(1, -1)).float().to(device)
#         for ep_d in ep_info:
#             if ep_d['ep_rewards'] is not None:
#                 print(f"reward = {round(ep_d['ep_rewards'], 2)} \t| ep_score = {round(600/ep_d['ep_len'], 2)}")
        predictions.append(a_t.item())        
        if done:
            break

Dirts Present..:6
[0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 0. 1.] 100 False
[0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 0. 1.] -1 False
[0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1.] 100 False
[0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1.] -1 False
[0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1.] -1 False
[0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1.] -1 False
[0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1.] -1 False
[0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1.] 100 False
[0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1.] -1 False
[0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1.] -1 False
[0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1.] 100 False
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1.] -1 False
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.

In [6]:
import pandas as pd

In [7]:
p = pd.DataFrame.from_dict({'actions': predictions, 'action_meanings':[env.action_meanings[i] for i in predictions]})
p['action_meanings'].value_counts().reset_index()

,index,action_meanings
0,move,7
1,clean,6
2,turn_left,3
3,turn_right,2
